In [1]:
%matplotlib qt
from scipy.ndimage.measurements import label, find_objects
from scipy.interpolate import LinearNDInterpolator, CloughTocher2DInterpolator, griddata
from scipy.spatial import Delaunay

import numpy as np
import matplotlib.pyplot as plt

In [2]:
cd "C:\Users\jdavies\Google Drive\MAGPIE\data\2015\s1029_15 Reconnection C 32 Wire TS in layer 280 ns"

C:\Users\jdavies\Google Drive\MAGPIE\data\2015\s1029_15 Reconnection C 32 Wire TS in layer 280 ns


In [3]:
class InterferogramInterpolator:
    def __init__(self, fn):
        i=plt.imread(fn)
        self.ysize=i.shape[0]
        self.xsize=i.shape[1]

        i=i.sum(axis=2)
        u=np.unique(i) #unique values in i - see what shade gray is, and also check for three colour
        if len(u) is not 3:
            print('Error: use only black, white and gray in interferogram')
        mask_colour=u[1]#find the shade of gray
        i=np.ma.masked_equal(i, mask_colour) #get the mask where gray is
        self.mask=i.mask
        i=(-i+4)/3 #this converts white to 0 and black to 1.
        i=np.clip(i,a_min=0, a_max=2) #clip
         # find fringes:
        structure=np.ones((3,3)) #look forconnectivity of pixels in every direction
        labeled_array, self.num_fringes = label(i, structure=structure)
        
        self.slices=find_objects(labeled_array)
        ##next two lines set white to be a large negative number so we don't confuse it with a fringe
        labeled_array[labeled_array==0]=-9999
        labeled_array=labeled_array-1 #two steps necessary to convert one of the array labels to 0 so we can use 0 indexing
        
        self.labeled_array=labeled_array
        
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111)
        self.ax.set_title('Dbl click to start, click to add, dbl click to end')

        cmap = plt.cm.jet
        cmap.set_under('w')
        self.im = self.ax.imshow(labeled_array, cmap=cmap, interpolation='nearest')
        self.ax.set_ylim([0,self.ysize])
        self.ax.set_xlim([0,self.xsize])

        self.fringe_dict={key: None for (key, value) in zip(range(self.num_fringes),range(self.num_fringes))}
        self.line,=self.im.axes.plot([0],[0],lw=1)
        self.fringe_indexes= self.im.get_array().copy()
        self.fringe_labels=np.zeros_like(self.fringe_indexes)-10000
        self.initialise_fringe_map()
        self.xs = []
        self.ys = []
        self.in_a_line=False
        self.cid = self.im.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        ax=self.ax
        print('click', event)
        if event.inaxes!=ax: 
            print('return')
            return
        if event.dblclick is True and self.in_a_line is False:
            print('Begin Line')
            ax.set_title('Dbl click to start, click to add, dbl click to end')
            self.in_a_line=True
            self.xs=[event.xdata]
            self.ys=[event.ydata]
            return
        if event.dblclick is True and self.in_a_line is True:
            print('End Line')
            self.in_a_line=False
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
            self.line.set_data(self.xs, self.ys)
            ax.set_ylim([0,self.ysize])
            ax.set_xlim([0,self.xsize])
            self.get_labels()
            self.check_labels()
            self.label_fringes()
            self.update_fringe_map()
            self.line.figure.canvas.draw()
            return
        if event.dblclick is False and self.in_a_line is True:
            print('Add segment')
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
            self.line.set_data(self.xs, self.ys)
            ax.set_ylim([0,self.ysize])
            ax.set_xlim([0,self.xsize])
            self.line.figure.canvas.draw()
            return
        
    def get_labels(self):
        d=self.fringe_indexes
        self.fringe_indexes_on_line=[]
        for i in range(len(self.xs)-1):
            x0, y0 = self.xs[i], self.ys[i] # These are in _pixel_ coordinates!!
            x1, y1 = self.xs[i+1], self.ys[i+1]
            length = int(np.hypot(x1-x0, y1-y0))
            x, y = np.linspace(x0, x1, length), np.linspace(y0, y1, length)
            # Extract the values along the line
            zi = d[y.astype(np.int), x.astype(np.int)]
            zi=[z for z in zi if z!=-10000]
            self.fringe_indexes_on_line.extend(zi)
            
    def check_labels(self):        
        fn=self.fringe_indexes_on_line
        #remove adjacent duplicates
        self.fringe_indexes_on_line=[n for i, n in enumerate(fn) if i==0 or n != fn[i-1]]
        #check there are no other duplicates
        if len(set(self.fringe_indexes_on_line)) is not len(self.fringe_indexes_on_line):
            self.im.axes.set_title('Line recrosses fringes, try again')
            self.im.axes.title.set_color('r')
            self.line.figure.canvas.draw()
            #self.fringe_indexes_on_line=[]
            return
        else:
            return
        
    def label_fringes(self):
        mode=+1
        start_index=self.fringe_dict[self.fringe_indexes_on_line[0]]
        if start_index is None:
            start_index=0
        for ind, f in enumerate(self.fringe_indexes_on_line):
            self.fringe_dict[f]=start_index+ind*mode
            
    def update_fringe_map(self):
        for f in range(self.num_fringes):
            s=self.slices[f]
            si=self.fringe_indexes[s]
            sl=self.fringe_labels[s]
            if self.fringe_dict[f] is not None:
                sl[si==f]=self.fringe_dict[f]
            else:
                sl[si==f]=1
        self.im.set_data(self.fringe_labels)
        
        mi=min(x for x in self.fringe_dict.values() if x is not None)
        ma=max(x for x in self.fringe_dict.values() if x is not None)
        self.im.set_clim([mi, ma])
        self.im.figure.canvas.draw()
    
    def initialise_fringe_map(self):
        for f in range(self.num_fringes):
            s=self.slices[f]
            si=self.fringe_indexes[s]
            sl=self.fringe_labels[s]
            sl[si==f]=1
        self.im.set_data(self.fringe_labels)
        self.im.set_clim([0, 1])
        self.im.figure.canvas.draw()
        
    def output_points_values(self):
        points=np.zeros((0,2))
        values=np.zeros((0))
        mi=min(x for x in self.fringe_dict.values() if x is not None)
        ma=max(x for x in self.fringe_dict.values() if x is not None)
        for fl in range(mi, ma+1):
            p=np.transpose(np.where(self.fringe_labeled_only==fl))
            points=np.append(points,p,axis=0)
            v=np.zeros(p.shape[0])+fl
            values=np.append(values, v)
        self.labeled_points=points
        self.labeled_values=values
        return points, values
        
    def remove_unlabeled(self):
        arr=np.zeros_like(self.fringe_labels)-10000
        for f in range(self.num_fringes):
            s=self.slices[f]
            si=self.fringe_indexes[s]
            sl=arr[s]
            if self.fringe_dict[f] is not None:
                sl[si==f]=self.fringe_dict[f]
        self.fringe_labeled_only=arr
        
    def interpolate(self, method='CT'):
        self.remove_unlabeled()
        self.points, self.values=self.output_points_values()
        grid_x, grid_y = np.mgrid[0:self.fringe_labeled_only.shape[0], 0:self.fringe_labeled_only.shape[1]]
        
        if method is 'CT':
            self.CT=CloughTocher2DInterpolator(self.points, self.values, tol=1e-8, maxiter=1000)
            self.d=self.CT(grid_x.ravel(), grid_y.ravel())
        if method is 'Linear':
            self.li=LinearNDInterpolator(self.points, self.values)
            self.d=self.li(grid_x.ravel(), grid_y.ravel())
            
        dr=self.d.reshape(grid_x.shape[0],grid_x.shape[1])
        drm=np.ma.asarray(dr)
        drm.mask=self.mask
        self.phase=drm

In [4]:
fn='s1029_15 532 plasma.png'
IP=InterferogramInterpolator(fn)

click MPL MouseEvent: xy=(116,232) xydata=(344.903225806,1516.93548387) button=1 dblclick=False inaxes=Axes(0.125,0.154114;0.775x0.691771)
click MPL MouseEvent: xy=(116,232) xydata=(344.903225806,1516.93548387) button=1 dblclick=True inaxes=Axes(0.125,0.154114;0.775x0.691771)
Begin Line
click MPL MouseEvent: xy=(535,238) xydata=(4359.19354839,1574.41935484) button=1 dblclick=False inaxes=Axes(0.125,0.154114;0.775x0.691771)
Add segment
click MPL MouseEvent: xy=(535,238) xydata=(4359.19354839,1574.41935484) button=1 dblclick=True inaxes=Axes(0.125,0.154114;0.775x0.691771)
End Line


In [5]:
IP.interpolate(method='Linear')

In [44]:
fn='s1029_15 532 vacuum.png'
IV=InterferogramInterpolator(fn)

In [46]:
IV.interpolate(method='Linear')

In [81]:
from scipy.spatial import Delaunay

tr=Delaunay(IP.points)
fig = plt.figure()
ax = fig.add_subplot(111)

ax.triplot(IP.points[:,1], IP.points[:,0], tr.simplices.copy())
#ax.plot(IP.points[:,1], IP.points[:,0], 'o')
ax.invert_yaxis()
im = ax.imshow(IP.fringe_labeled_only, clim=[-100,100],interpolation='nearest')

## Code to remove flat triangles

In [36]:
from scipy.spatial import Delaunay
tr=Delaunay(IP.points) #initial Delaunay triangulation
values=IP.fringe_labeled_only #short hand

In [33]:
flat_tri=[]
for i,s in enumerate(tr.simplices):
    #if the length of the set is 1, all vertices have the same value and the triangle is flat. 
    #We just want the triangle index
    v=[values[pp[0],pp[1]] for pp in tr.points[s]]
    if v[1:] == v[:-1]:
        flat_tri.append(True)
    else:
        flat_tri.append(False)

## Code using edge flipping:

In [41]:
for j, ft in enumerate(flat_tri[:10000]):
    if ft is True:
        ne=tr.neighbors[j]
        for n in ne:
            if flat_tri[n] is False:
                sim0=tr.simplices[i]
                sim1=tr.simplices[n]
                for i,p in enumerate(sim0):
                    if p not in sim1:
                        p0=p
                        p0i=i
                for i,p in enumerate(sim1):
                    if p not in sim0:
                        p1=p
                        p1i=i
                sim0[p0i]=p1
                sim1[p1i]=p0       

In [42]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.triplot(IP.points[:,1], IP.points[:,0], tr.simplices.copy())
#ax.plot(IP.points[:,1], IP.points[:,0], 'o')
ax.invert_yaxis()
im = ax.imshow(IP.fringe_labeled_only, clim=[-100,100],interpolation='nearest')

## Code intended to remove flat triangles using critical points: SLOW!

In [51]:
def point_distance(p0,p1):
    return np.sqrt((p0[0]-p1[0])**2+(p0[1]-p1[1])**2)#np.linalg.norm(np.asarray(p0)-np.asarray(p1))

#if points don't touch in a NESW sense, they are no adjacent.
#for a flat triangle form on a close packed contour there is only one pair of non adjacent points
#so we get a speed up by sequentially considering each pair
def non_adjacent_points(p):
    if point_distance(p[0],p[1])>1:
        return p[0], p[1]
    if point_distance(p[1],p[2])>1:
        return p[1], p[2]
    if point_distance(p[2],p[0])>1:
        return p[2], p[0]
    else:
        return None
    
def search_in_direction_first_n_not(point, array, direction, n, n_not):
    '''
    Searches for the nearest n points to a specified point in an array
    Valid points are not equal to a given number
    This is for searching a sparse contour map for non-'background' elements
    '''
    n_found=[]
    c=point
    c0max=array.shape[0]
    c1max=array.shape[1]
    while len(n_found)<n:
        c=(c[0]+direction[0], c[1]+direction[1])
        if c[0]>=c0max or c[0]<0:
            return n_found
        if c[1]>=c1max or c[1]<0:
            return n_found
        if array[c]!=n_not:
            n_found.append(c)
    return n_found

def search_vertical(point, array, n, nth_not):
    line=array[:,point[1]]
    n_found=[]
    valid=np.where(line!=nth_not)[0]
    while len(n_found)<n:
        n_found.append((np.where()[0],point[1]))
    (find_nth_index_not(pp, line, 1, -10000), point[1])

def parabolic_interpolation(p0,p1,p2, pc, array):
    #parabolic interpolation using three points.
    h0=values[p0]
    h1=values[p1]
    h2=values[p2]

    degenerate=h0 is h1 and h1 is h2 #can't interpolate if all points are on the same contour

    if not degenerate:
        sc=point_distance(pc, p0)
        s0=0 #by definition
        s1=point_distance(p1, p0)
        s2=point_distance(p2, p0)
        term0=h0*((sc-s1)/(s0-s1))*((sc-s2)/(s0-s2))
        term1=h1*((sc-s0)/(s1-s0))*((sc-s2)/(s1-s2))
        term2=h2*((sc-s0)/(s2-s0))*((sc-s1)/(s2-s1))
        hc=term0+term1+term2
        return hc
    if degenerate:
        return None

directions=[(1,0),(1,1),(0,1),(-1,1)]

def estimate_new_point_value(point, value):
    hcs=[]
    for d in directions:
        rev_d=(-d[0],-d[1])
        fwd=search_in_direction_first_n_not(point, values, d, 2, n_not=-10000)
        rev=search_in_direction_first_n_not(point, values, rev_d, 2, n_not=-10000)
        if len(fwd) is 2 and len(rev)>=1:
            f_hc=parabolic_interpolation(fwd[1],fwd[0],rev[0], point,values)
            if f_hc is not None:
                hcs.append(f_hc)
        if len(rev) is 2 and len(fwd)>=1:
            r_hc=parabolic_interpolation(fwd[0],rev[1],rev[0],point,values)
            if r_hc is not None:
                hcs.append(r_hc)
    if len(hcs)>0:
        return np.mean(hcs)
    else:
        return None

In [29]:
BT_edge_points=[]
#find the Bridge/Tunnel edges (B/T) edges of the flat triangles
#these are the edges which join non-adjacent points
flat_triangle_points=tr.points[tr.simplices[flat_tri]]
for ft in flat_triangle_points:
    edge=non_adjacent_points(ft)
    if edge is not None:
        BT_edge_points.append(edge)

In [30]:
new_p=[]
for bte in BT_edge_points:
    #the critical points to be added are midpoints of the B/T edges
    n_p=(round((bte[1][0]+bte[0][0])/2),round((bte[1][1]+bte[0][1])/2))
    new_p.append(n_p)

In [52]:
hcs=[estimate_new_point_value(p, values) for p in new_p[:1000]]

In [299]:
def search_in_direction_first_n_not(point, array, direction, n, n_not):
    n_found=[]
    c=point
    c0max=array.shape[0]
    c1max=array.shape[1]
    while len(n_found)<n:
        c=(c[0]+direction[0], c[1]+direction[1])
        if c[0]>=c0max or c[0]<0:
            return n_found
        if c[1]>=c1max or c[1]<0:
            return n_found
        if array[c]!=n_not:
            n_found.append(c)
    return n_found
            

In [315]:
hcs
point=new_p[5000]
hcs=[]

##vertical sweep to smaller y values
line=values[:,point[1]] #identify a line along which to search, here a vertical line
pp=point[0]
#find three points for parabolic interpolation
p2=(find_nth_index_not(pp, line, 1, -10000), point[1]) #search up the array
p1=(find_nth_index_not(pp, line, -1, -10000), point[1]) #search up the array
p0=(find_nth_index_not(pp, line, -2, -10000), point[1]) #search up the array
hcs.append(parabolic_interpolation(p0,p1,p2,values))

##vertical sweep to larger y values
#find thre points for parabolic interpolation
p2=(find_nth_index_not(pp, line, -1, -10000), point[1]) #search up the array
p1=(find_nth_index_not(pp, line, 1, -10000), point[1]) #search up the array
p0=(find_nth_index_not(pp, line, 2, -10000), point[1]) #search up the array
hcs.append(parabolic_interpolation(p0,p1,p2,values))

##horizontal sweep to smaller x values
line=values[point[0],:] #identify a line along which to search, here a vertical line
pp=point[1]
#find three points for parabolic interpolation
p2=(point[0], find_nth_index_not(pp, line, 1, -10000)) #search up the array
p1=(point[0],find_nth_index_not(pp, line, -1, -10000)) #search up the array
p0=(point[0],find_nth_index_not(pp, line, -2, -10000)) #search up the array
hcs.append(parabolic_interpolation(p0,p1,p2,values))

##vertical sweep to larger y values
#find thre points for parabolic interpolation
p2=(point[0],find_nth_index_not(pp, line, -1, -10000)) #search up the array
p1=(point[0],find_nth_index_not(pp, line, 1, -10000)) #search up the array
p0=(point[0],find_nth_index_not(pp, line, 2, -10000)) #search up the array
hcs.append(parabolic_interpolation(p0,p1,p2,values))

[59.714458560193577,
 59.744958949096755,
 60.055746258421784,
 59.967670627245099]

In [281]:
(1,1)+(1,1)

(1, 1, 1, 1)

fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.cm.jet
cmap.set_under('w')

im = ax.imshow(IP.phase, cmap=cmap,interpolation='nearest')

In [30]:
tr.simplices.copy()

array([[     3,      2,     18],
       [    15,      3,     18],
       [134443, 134445, 134456],
       ..., 
       [ 55347,  53242,  55348],
       [ 23279,  23280,  23282],
       [ 23277,  23279,  23282]], dtype=int32)

In [32]:
from matplotlib import tri
t=tri.Triangulation(IP.points[:,1], IP.points[:,0])

In [1]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.triplot(t.x,t.y,t.triangles)
ax.plot(t.x, t.y, 'o')
ax.invert_yaxis()


NameError: name 'plt' is not defined

In [15]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(d[1740,:])

In [12]:
grid_x, grid_y = np.mgrid[0:IP.fringe_labeled_only.shape[0], 0:IP.fringe_labeled_only.shape[1]]
d=griddata(IP.points, IP.values, (grid_x,grid_y))

In [14]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.cm.jet
cmap.set_under('w')

im = ax.imshow(d, cmap=cmap,interpolation='nearest')

In [9]:
grid_x.ravel()

array([   0,    0,    0, ..., 3167, 3167, 3167])

In [10]:
?griddata

In [124]:
t=plt.imread('s1029_15 532 plasma.png')
t=t.sum(axis=2)
mask_colour=np.unique(t)[1]#find the shade of gray
tm=np.ma.masked_equal(t, mask_colour) #get the mask
tt=(-tm+4)/3
th=np.clip(tt,a_min=0, a_max=2)

In [125]:
structure=np.ones((3,3))
labeled_array, num_fringes = label(th, structure=structure)
slices=find_objects(labeled_array)
labeled_array[labeled_array==0]=-9999
labeled_array=labeled_array-1
#la=np.ma.masked_equal(labeled_array, 0)

In [276]:
class FringeNumberer:
    def __init__(self, im, slices):
        self.num_fringes=len(slices)
        self.fringe_dict={key: None for (key, value) in zip(range(num_fringes),range(num_fringes))}
        self.line,=im.axes.plot([0],[0],lw=1)
        self.im = im
        self.fringe_indexes= im.get_array().copy()
        self.fringe_labels=np.zeros_like(self.fringe_indexes)-10000
        self.initialise_fringe_map()
        self.xs = []
        self.ys = []
        self.in_a_line=False
        self.cid = im.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        ax=self.im.axes
        print('click', event)
        if event.inaxes!=ax: 
            print('return')
            return
        if event.dblclick is True and self.in_a_line is False:
            print('Begin Line')
            ax.set_title('Dbl click to start, click to add, dbl click to end')
            self.in_a_line=True
            self.xs=[event.xdata]
            self.ys=[event.ydata]
            return
        if event.dblclick is True and self.in_a_line is True:
            print('End Line')
            self.in_a_line=False
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
            self.line.set_data(self.xs, self.ys)
            ax.set_xlim([0,la.shape[0]])
            ax.set_xlim([0,la.shape[1]])
            self.get_labels()
            self.check_labels()
            self.label_fringes()
            self.update_fringe_map()
            self.line.figure.canvas.draw()
            return
        if event.dblclick is False and self.in_a_line is True:
            print('Add segment')
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
            self.line.set_data(self.xs, self.ys)
            ax.set_xlim([0,la.shape[0]])
            ax.set_xlim([0,la.shape[1]])
            self.line.figure.canvas.draw()
            return
        
    def get_labels(self):
        d=self.fringe_indexes
        self.fringe_indexes_on_line=[]
        for i in range(len(self.xs)-1):
            x0, y0 = self.xs[i], self.ys[i] # These are in _pixel_ coordinates!!
            x1, y1 = self.xs[i+1], self.ys[i+1]
            length = int(np.hypot(x1-x0, y1-y0))
            x, y = np.linspace(x0, x1, length), np.linspace(y0, y1, length)
            # Extract the values along the line
            zi = d[y.astype(np.int), x.astype(np.int)]
            zi=[z for z in zi if z!=-10000]
            self.fringe_indexes_on_line.extend(zi)
            
    def check_labels(self):        
        fn=self.fringe_indexes_on_line
        #remove adjacent duplicates
        self.fringe_indexes_on_line=[n for i, n in enumerate(fn) if i==0 or n != fn[i-1]]
        #check there are no other duplicates
        if len(set(self.fringe_indexes_on_line)) is not len(self.fringe_indexes_on_line):
            self.im.axes.set_title('Line recrosses fringes, try again')
            self.im.axes.title.set_color('r')
            self.line.figure.canvas.draw()
            #self.fringe_indexes_on_line=[]
            return
        else:
            return
        
    def label_fringes(self):
        mode=+1
        start_index=self.fringe_dict[self.fringe_indexes_on_line[0]]
        if start_index is None:
            start_index=0
        for ind, f in enumerate(self.fringe_indexes_on_line):
            self.fringe_dict[f]=start_index+ind*mode
            
    def update_fringe_map(self):
        for f in range(self.num_fringes):
            s=slices[f]
            si=self.fringe_indexes[s]
            sl=self.fringe_labels[s]
            if self.fringe_dict[f] is not None:
                sl[si==f]=self.fringe_dict[f]
            else:
                sl[si==f]=1
        self.im.set_data(self.fringe_labels)
        
        mi=min(x for x in self.fringe_dict.values() if x is not None)
        ma=max(x for x in self.fringe_dict.values() if x is not None)
        self.im.set_clim([mi, ma])
        self.im.figure.canvas.draw()
    
    def initialise_fringe_map(self):
        for f in range(self.num_fringes):
            s=slices[f]
            si=self.fringe_indexes[s]
            sl=self.fringe_labels[s]
            sl[si==f]=1
        self.im.set_data(self.fringe_labels)
        self.im.set_clim([0, 1])
        self.im.figure.canvas.draw()
        
    def output_points_values(self):
        points=np.zeros((0,2))
        values=np.zeros((0))
        mi=min(x for x in self.fringe_dict.values() if x is not None)
        ma=max(x for x in self.fringe_dict.values() if x is not None)
        for fl in range(mi, ma+1):
            p=np.transpose(np.where(self.fringe_labelled_only==fl))
            points=np.append(points,p,axis=0)
            v=np.zeros(p.shape[0])+fl
            values=np.append(values, v)
        self.labeled_points=points
        self.labeled_values=values
        return points, values
        
    def remove_unlabelled(self):
        arr=np.zeros_like(self.fringe_labels)-10000
        for f in range(self.num_fringes):
            s=slices[f]
            si=self.fringe_indexes[s]
            sl=arr[s]
            if self.fringe_dict[f] is not None:
                sl[si==f]=self.fringe_dict[f]
        self.fringe_labelled_only=arr        
            
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('Dbl click to start, click to add, dbl click to end')

cmap = plt.cm.jet
cmap.set_under('w')
im = ax.imshow(labeled_array, cmap=cmap, interpolation='nearest')

FN = FringeNumberer(im, slices)

plt.show()

click MPL MouseEvent: xy=(199,227) xydata=(439.516129032,1645.16129032) button=1 dblclick=False inaxes=Axes(0.125,0.154114;0.775x0.691771)
click MPL MouseEvent: xy=(199,227) xydata=(439.516129032,1645.16129032) button=1 dblclick=True inaxes=Axes(0.125,0.154114;0.775x0.691771)
Begin Line
click MPL MouseEvent: xy=(522,231) xydata=(4346.77419355,1596.77419355) button=1 dblclick=False inaxes=Axes(0.125,0.154114;0.775x0.691771)
Add segment
click MPL MouseEvent: xy=(522,231) xydata=(4405.28870293,1583.68200837) button=1 dblclick=True inaxes=Axes(0.157585,0.1;0.70983x0.8)
End Line


In [277]:
FN.remove_unlabelled()
points, values=FN.output_points_values()

In [294]:
%time
li=LinearNDInterpolator(points, values)
d=li(grid_x.ravel(), grid_y.ravel())

Wall time: 0 ns


In [295]:
%time
CT=CloughTocher2DInterpolator(points, values, tol=1e-8, maxiter=1000)
d=CT(grid_x.ravel(), grid_y.ravel())

Wall time: 0 ns


In [279]:
grid_x, grid_y = np.mgrid[0:labeled_array.shape[0], 0:labeled_array.shape[1]]

In [280]:
d=CT(grid_x.ravel(), grid_y.ravel())

In [281]:
dr=d.reshape(grid_x.shape[0],grid_x.shape[1])

fig = plt.figure()
ax = fig.add_subplot(111)
im = ax.imshow(np.nan_to_num(dr), clim=[-100,100],interpolation='nearest')

In [282]:
fl=FN.fringe_labelled_only

In [249]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.cm.jet
cmap.set_under('w')

im = ax.imshow(fl[1200:2200,1500:3500], cmap=cmap,clim=[0,fl.max()],interpolation='nearest')

In [262]:
from scipy.spatial import Delaunay

In [284]:
tri=Delaunay(points)
fig = plt.figure()
ax = fig.add_subplot(111)

ax.triplot(points[:,1], points[:,0], tri.simplices.copy())
ax.plot(points[:,1], points[:,0], 'o')
ax.invert_yaxis()
im = ax.imshow(np.nan_to_num(dr), clim=[-100,100],interpolation='nearest')


In [283]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.cm.jet
cmap.set_under('w')

#ax.triplot(points[:,1], points[:,0], tri.simplices.copy())
ax.plot(points[:,1], points[:,0], 'o')
ax.invert_yaxis()

im = ax.imshow(fl, cmap=cmap,clim=[0,fl.max()],interpolation='nearest')

In [286]:
drm=np.ma.asarray(dr)
drm.mask=tm.mask

In [291]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.cm.jet
cmap.set_under('w')

im = ax.imshow(drm, cmap=cmap,interpolation='nearest')

In [289]:
drm.max()

nan